In [ ]:
!pip install bs4
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1270 sha256=66d35a95256202ae42b2483316de3031c0fa33e6a9c4926fdd6eef743fdc0ecf
  Stored in directory: /root/.cache/pip/wheels/73/2b/cb/099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
Successfully built bs4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

## 크롤링 코드 

베스트 소설도 아래 링크로 연결 가능

In [ ]:
def crawlingNovel(page):
  URL = "https://novel.naver.com/best/detail?novelId="
  number = "&volumeNo="
  smallData=[]
  for c in range(1, 200):
    text=""
    starCount=""
    if(len(smallData)>=10):
      break;
    pageURL=URL+page+number+str(c)
    html = requests.get(pageURL).text # html 문서 전체를 긁어서 출력해줌, .text는 태그 제외하고 text만 출력되게 함
    padeBsObject = BeautifulSoup(html, 'html.parser')
    try:
      text=padeBsObject.find('div', {'class': "detail_view_content ft15"}).string
      starCount = padeBsObject.find('span', {'class': "num"}, {"id":"currentStarScoreCount"}).string.replace("명", "") #별점수
      smallData.append(text)
      smallData.append(starCount)
    except:
      continue

  return smallData

In [ ]:
def crawlingPage(URL):
  PAGE_URL = URL + '&page=1000' #가장 맨 뒤 페이지로 이동
  html = requests.get(PAGE_URL).text # html 문서 전체를 긁어서 출력해줌, .text는 태그 제외하고 text만 출력되게 함
  bsObject_page = BeautifulSoup(html, 'html.parser')
  totalData=[]

  rating=bsObject_page.find_all("p", {"class": "rating"})
  b =[]

  for i in rating:
    if((re.search(':', i.get_text()))== None and (re.search('\d{4}',i.get_text()) == None)): #올린 날짜가 없는 추천 정보 제거
      break
    b=b+[i.get_text()]


  if len(b)>5:
    b=b[-5:]
  elif len(b)<5:
    a=[]
    c = 5-len(i)
    page_count =bsObject_page.find_all('a', {'class': "NPI=a:page"}) 
    PAGE_URL="https://novel.naver.com"+page_count[-1]["href"]
    html_1 = requests.get(PAGE_URL).text 
    bsObject_page_1 = BeautifulSoup(html_1, 'html.parser')
    rating_1=bsObject_page_1.find_all("p", {"class": "rating"})
    for i in rating_1:
      if((re.search(':', i.get_text()))== None and (re.search('\d{4}',i.get_text()) == None)): #올린 날짜가 없는 추천 정보 제거
        break
      a=a+[i.get_text()]

    b=a[-5+len(b):]+b



  some_data =[]
  for i in b:
    i=i.split()
    del i[1:4]
    i[0]= i[0].replace('별점', '')
    some_data.append(i)

    for i in some_data:
      for j in i:
        totalData.append(j)

  return totalData[-15:]


In [ ]:
def crawling(URL):
  html = requests.get(URL).text # html 문서 전체를 긁어서 출력해줌, .text는 태그 제외하고 text만 출력되게 함
  bsObject = BeautifulSoup(html, 'html.parser')
  data_name = ["이름", "장르", "총합 별점", "관심수", "작품리뷰수", "1화", "1화 별점수", "2화", "2화 별점수", "3화", "3화 별점수", "4화", "4화 별점수", "5화", "5화 별점수", "5화 별점", "5화 조회수", "5화 올린 날짜", "4화 별점", "4화 조회수", "4화 올린 날짜", "3화 별점", "3화 조회수", "3화 올린 날짜", "2화 별점", "2화 조회수", "2화 올린 날짜", "1화 별점", "1화 조회수", "1화 올린 날짜"]
  Data=[]

  title=bsObject.find('h2', {'class': "title"}).get_text()
  genre= bsObject.find('span', {'class': "item"}).get_text()
  totalStar = bsObject.find('span', {'class': "score_area"}).get_text().replace('별점', "")
  concern=bsObject.find('span', {'id':'concernCount'}).get_text()
  review=bsObject.find('a', {'class':'review'}).get_text()

  Data=[title, genre, totalStar, concern, review]

  pageNum = re.sub(r"[^0-9]", "", URL)
  smallData=crawlingNovel(pageNum)
  if len(smallData)>10:
    del smallData[10:12]

  Data=Data+smallData
  Data= Data+crawlingPage(URL)

  return pd.DataFrame([Data], columns = data_name)


링크와 댓글 자료를 모아둔 빅데이터 자료

In [ ]:
best = pd.read_excel('/content/빅데이터 데이터 모음.xlsx')
best

,장르,링크,1화 댓글,2화 댓글,3화 댓글,4화 댓글,5화 댓글
0,로멘스,https://novel.naver.com/best/list?novelId=1087448,"불쌍하다고 말한 그 순간부터 업보 시작이쥬 ,,, 둘의 관계은 뭔지, 무슨 일이 있...",아니… 가은아 연기는 아닌가봐… 너무 어설퍼서 다보옄ㅋㅋㅠㅠ 똑똑한 남자인걸 알면서...,아 짜릿ㅎㅎㅎ 역시 집착 왕창 스푼으로 넣어줘야죠! 거기에 도망가는 여주… 아무래도...,"안녕하세요, 엄신주 작가님. 로맨스 웹소설 출판사 스노우미디어입니다. 작가님께 연락...",설정이 너무 약해요…기억상실이면 이렇게 못할텐데…/아슬아슬~~ 뭔가 어설픔이 더 불...
1,로멘스,https://novel.naver.com/best/list?novelId=1093077,라인에 이어 ~\n법 스토리 너무 좋아요 ~\n쭉 ~ 열심히 따라 다니겠습니다 ~/...,현백합.. 이름부터 뭔가 불안한데요.. 마지막에는 임신이라니..\n회장님도 그렇고 ...,댑악...이번 소설은 더 재미있는 것 같은데요??/와 현백합... 벌써부터 화딱지나...,도와줘/잘 읽고 갑니다 ~/그 엄마에 그 딸이네요 정말... 제국이가 하인이에게 그...,잘 타이르네/제니가 혼이 나서 좀 사이다인데 하인이 말처럼 나중에 제니가 일으킬 후...
2,로멘스,https://novel.naver.com/best/list?novelId=1089861,작가분 경험담인줄/정주행 시작합니다~^^/초반 시작이 너무 서정적....\n너무 무...,어떤약속도 없이 미국으로 떠난다면서 그걸 첫관계이후에 말해? 양심은 밥말아먹었나 그...,아 이런 쓰레귀가…/이기적 남자..../참으로 무책임한 남자일세./한국뜨기 일년전부...,미안함과 죄책감\nG R 를해라 G R 를\n남주 따로 있는거죠?/완전 미ㅇㅇ 식히...,평범한 소설이지만 계속보게되네요.결론도 다른소설처럼 되나요?같은부류 흙수저끼리결혼?...
3,로멘스,https://novel.naver.com/best/list?novelId=1095662,시작은 프롤로그여요~\n끝이 에필로그,재밌게 보고 있어요,도현에게 무슨일이 있었는지 넘 궁금해요,NaN,1일1연재가 시급해요 작가님 자주 와주세요
4,로멘스,https://novel.naver.com/best/list?novelId=1090566,앗~~~\n만나서 반가워요~~~\n즐겁고 행복하게 볼께요~/아...남에겐 차가워도 ...,모르면 용감하다더니..ㅎㅎㅎ/ㅁㅊ....므가 이래/ㅋㅋㅋㅋ/옴마나. 얘네 진도봐라?...,엇..지완이에게 마음에 둔 여자가 있었네요.\n세라가 알게 되면 상처받진 않을지 걱...,왠지 AI 랑 연애하는거 같아/달달한데도 뭔가 불안해요.\n지완이 첫사랑도 다른 사...,아하. 이 이야기는 다른 작품의 후속이었군요.\n찾아 읽어봐야겠어요ㅋ\n박터지게 싸...
...,...,...,...,...,...,...,...
135,라이트노벨,https://novel.naver.com/best/list?novelId=896722,진짜네..이거 귀환한 마법사는 특별해야합니다랑 비슷하네\n\n시작은 뭐 비슷비슷 하...,쎄다!\n기피1순위 ㄱ\n\n설정은 몰라도 대사까지 베끼는 것은 오바인데...\n\...,지루하다\n\n잘보고 갑니다,주인공...긍정 그 자체넹..\n\n시발 ㅋㅋㅋ 그냥 개똑같잖아 ㅋㅋㅋ\n\n누가봐...,NaN
136,라이트노벨,https://novel.naver.com/best/list?novelId=1050583,NaN,NaN,NaN,NaN,NaN
137,라이트노벨,https://novel.naver.com/best/list?novelId=763126,NaN,NaN,NaN,NaN,NaN
138,라이트노벨,https://novel.naver.com/best/list?novelId=659886,재밌군요! 다만 한 가지 아쉬운 부분이... 문장들\n이 너무 붙어 있어서 읽는 도...,NaN,정주행 ~ 정주행 ~\n과 함께 평점과 하트를 누르며\n정주행 ~ 정주행 ~\n\n...,NaN,작가님. 학원물도괜찮아요.\n이제 저는 평범한 일상을 보내려 합니다.\n작가님 작품...


In [ ]:
best_url=best['링크']

In [ ]:
Data = pd.DataFrame()
for i in best_url: 
  a=crawling(i)
  #print(i)
  Data= pd.concat([Data, a], ignore_index=True)

https://novel.naver.com/best/list?novelId=1087448
https://novel.naver.com/best/list?novelId=1093077
https://novel.naver.com/best/list?novelId=1089861
https://novel.naver.com/best/list?novelId=1095662
https://novel.naver.com/best/list?novelId=1090566
https://novel.naver.com/best/list?novelId=1094121
https://novel.naver.com/best/list?novelId=1094370
https://novel.naver.com/best/list?novelId=1086178
https://novel.naver.com/best/list?novelId=1098270
https://novel.naver.com/best/list?novelId=1086009
https://novel.naver.com/best/list?novelId=1095354
https://novel.naver.com/best/list?novelId=1089621
https://novel.naver.com/best/list?novelId=1095743
https://novel.naver.com/best/list?novelId=1092575
https://novel.naver.com/best/list?novelId=1098295
https://novel.naver.com/best/list?novelId=758021
https://novel.naver.com/best/list?novelId=1096437
https://novel.naver.com/best/list?novelId=1096416
https://novel.naver.com/best/list?novelId=1088062
https://novel.naver.com/best/list?novelId=1089179
h

In [ ]:
Data

,이름,장르,총합 별점,관심수,작품리뷰수,1화,1화 별점수,2화,2화 별점수,3화,...,4화 올린 날짜,3화 별점,3화 조회수,3화 올린 날짜,2화 별점,2화 조회수,2화 올린 날짜,1화 별점,1화 조회수,1화 올린 날짜
0,임신 계략,로맨스,9.98,"2,179",4,1화\r\n\r\n가은은 꽉 닫힌 서재의 문을 한참동안이나 멍하니 서서 바라봤다. ...,166,2화\r\n\r\n“블랙박스 영상은 잘 처리하셨죠?”\r\n\r\n가은의 목소리가 ...,156,3화\r\n\r\n가은은 지도준의 손길에 이끌려 어쩔 수 없이 그의 방으로 들어올 ...,...,2023.01.02,10.0,2만,2022.12.31,10.0,3만,2022.12.30,9.95,3만,2022.12.29
1,제국의 하인,로맨스,9.99,"2,135",18,1. 나쁜X\r\n\r\n\r\n* * *\r\n\r\n사람들은 하인에게 피도 눈물...,225,2. 악연의 시작\r\n\r\n\r\n* * *\r\n\r\n하인의 아버지는 하인이...,181,3. 가스라이팅\r\n\r\n\r\n* * *\r\n\r\n현백합의 임신 소식에 해...,...,2023.02.06,9.96,2만,2023.02.05,9.95,2만,2023.02.05,10.0,3만,2023.02.05
2,"첫사랑, 첫이별, 첫아이",로맨스,9.99,"1,549",10,파르르 입술이 떨려왔다.\r\n\r\n\r\n내 호흡소리도 거칠어지는 듯 했다.\r...,211,그는 대답대신 침대에서 일어나 담배에 불을 붙이더니 창가로 갔다.\r\n\r\n그리...,172,임신진단키트의 선명한 빨간 두 줄이 생리의 붉은 출혈 마냥 선명하게 내 두 눈에 들...,...,2023.01.29,9.99,1만,2023.01.25,9.93,2만,2023.01.20,9.86,2만,2023.01.15
3,못난이 아내,로맨스,9.98,"1,136",0,"유난히도 개운한 월요일 아침이였다. \r\n\r\n\r\n “으음, 몇...",99,"“아저씨, 좋은 아침입니다.“\r\n\r\n\r\n 동그란 안경에 어깨...",100,\r\n\r\n “오늘은 바닐라 라떼로.“\r\n\r\n\r\n\r\n\r...,...,2023.02.26,9.93,1만,2023.02.25,9.94,1만,2023.02.24,9.92,1만,2023.02.22
4,남편의 체온,로맨스,9.97,"1,482",4,"\r\n#1. 연애다운 연애\r\n\r\n\r\n‘너, 나랑 연애하지 않을래?’\r...",177,#2. 키스 해봤니?\r\n\r\n\r\n저를 데리러 온 지완의 차에 올라탄 세라는...,155,\r\n#3. 다음에 만나면…… 안아주세요\r\n\r\n\r\n미처 반응할 새도 없...,...,2023.01.22,9.93,2만,2023.01.21,10.0,2만,2023.01.20,9.97,3만,2023.01.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,나만 빼고 각성자 - 천마귀환,현판,9.78,29,0,천무세계(天武世界). \r\n\r\n무(武)로서 하늘에 닿고자 하는 사람들이 사는 ...,14,한달. \r\n\r\n길다면 길고 짧다면 짧은 시간이다. \r\n\r\n그러나 안수...,15,정적. \r\n\r\n그리고 또 정적. \r\n\r\n종말급 괴수종과의 전투를 준비...,...,2022.12.25,9.83,817,2022.12.24,9.27,913,2022.12.24,9.21,"1,131",2022.12.24
76,돈의흐름,현판,10.0,15,0,이건 역사에 기록되지 않았던 돈의 시초의 관한 이야기이다. \r\n 옛날 아주 먼 ...,3,나는 천천히 그리고 조심스럽게 숙소가 있는 언덕을 올라갔다. \r\n 내가 먼저 저...,3,9시 30분 이제 약속했던 10시가 다되어 간다. 면접을 마치고 결과를 기다리는 취...,...,2021.10.09,0.0,590,2021.10.09,10.0,"1,144",2021.10.09,10.0,"2,146",2021.10.09
77,나만 보이는 시스템 창,현판,9.78,9,0,\r\n“뭐하냐! 피해!”\r\n“우왓!”\r\n\r\n콰앙!\r\n\r\n땅을 내...,6,\r\n*\r\n\r\n“어머 사도야 어서오렴”\r\n“안녕하세요.”\r\n\r\n...,8,"\r\n*\r\n\r\n“에이스 결정전까지 앞으로 6개월, 쉴 시간 같은 건 없다....",...,2022.08.14,9.33,590,2022.08.13,9.13,757,2022.08.12,9.33,"1,464",2022.08.11
78,듀얼 클래스 마스터,현판,9.14,339,9,"내 이름은 이태양(李太陽), 서울에 있는 한국대학교 상경대학 IT경영학과 25학번 ...",70,“왜 이런 일이 생겨난 것일까?”\r\n\r\n쉼터에서 복잡해진 생각을 정리하고자 ...,83,“후우… 후우…”\r\n\r\n동이 틀 때 까지 고블린들과 드잡이질을 한 나는 지친...,...,2019.06.12,9.54,"9,601",2019.06.12,9.33,1만,2019.06.12,9.41,2만,2019.06.12


In [ ]:
best_test=best_test.drop('링크', axis = 'columns')
best_test=best_test.drop('장르', axis = 'columns')

In [ ]:
best_test

,1화 댓글,2화 댓글,3화 댓글,4화 댓글,5화 댓글
0,"[불쌍하다고 말한 그 순간부터 업보 시작이쥬 ,,, 둘의 관계은 뭔지, 무슨 일이 ...",[아니… 가은아 연기는 아닌가봐… 너무 어설퍼서 다보옄ㅋㅋㅠㅠ 똑똑한 남자인걸 알면...,[아 짜릿ㅎㅎㅎ 역시 집착 왕창 스푼으로 넣어줘야죠! 거기에 도망가는 여주… 아무래...,"[안녕하세요, 엄신주 작가님. 로맨스 웹소설 출판사 스노우미디어입니다. 작가님께 연...","[설정이 너무 약해요…기억상실이면 이렇게 못할텐데…, 아슬아슬~~ 뭔가 어설픔이 더..."
1,[라인에 이어 ~\n법 스토리 너무 좋아요 ~\n쭉 ~ 열심히 따라 다니겠습니다 ~...,[현백합.. 이름부터 뭔가 불안한데요.. 마지막에는 임신이라니..\n회장님도 그렇고...,"[댑악...이번 소설은 더 재미있는 것 같은데요??, 와 현백합... 벌써부터 화딱...","[도와줘, 잘 읽고 갑니다 ~, 그 엄마에 그 딸이네요 정말... 제국이가 하인이에...","[잘 타이르네, 제니가 혼이 나서 좀 사이다인데 하인이 말처럼 나중에 제니가 일으킬..."
2,"[작가분 경험담인줄, 정주행 시작합니다~^^, 초반 시작이 너무 서정적....\n너...",[어떤약속도 없이 미국으로 떠난다면서 그걸 첫관계이후에 말해? 양심은 밥말아먹었나 ...,"[아 이런 쓰레귀가…, 이기적 남자...., 참으로 무책임한 남자일세., 한국뜨기 ...","[미안함과 죄책감\nG R 를해라 G R 를\n남주 따로 있는거죠?, 완전 미ㅇㅇ ...",[평범한 소설이지만 계속보게되네요.결론도 다른소설처럼 되나요?같은부류 흙수저끼리결혼...
3,[시작은 프롤로그여요~\n끝이 에필로그],[재밌게 보고 있어요],[도현에게 무슨일이 있었는지 넘 궁금해요],NaN,[1일1연재가 시급해요 작가님 자주 와주세요]
4,"[앗~~~\n만나서 반가워요~~~\n즐겁고 행복하게 볼께요~, 아...남에겐 차가워...","[모르면 용감하다더니..ㅎㅎㅎ, ㅁㅊ....므가 이래, ㅋㅋㅋㅋ, 옴마나. 얘네 진...",[엇..지완이에게 마음에 둔 여자가 있었네요.\n세라가 알게 되면 상처받진 않을지 ...,"[왠지 AI 랑 연애하는거 같아, 달달한데도 뭔가 불안해요.\n지완이 첫사랑도 다른...",[아하. 이 이야기는 다른 작품의 후속이었군요.\n찾아 읽어봐야겠어요ㅋ\n박터지게 ...
...,...,...,...,...,...
75,[통일대한민국님... 무협 고수시군요. 심후한 내공이 돋보입니다 ㅋㅋㅋ 댓글 감사합...,[무극여의경에 도달한 판타지 주인공들이 있는 소설들은 몆개 읽어봐지만\n무협에서 무...,"[자연경,초신경,??경,무극여의경\n여기 주인공 어디까지\n잘보고 갑니다]",[잘보고 갑니다],[돌+아이 다크히어로 주인공...\n잘보고 갑니다]
76,NaN,NaN,NaN,NaN,NaN
77,NaN,NaN,NaN,NaN,NaN
78,"[음...\n잘보고 갑니다, 2021.6.19 시작, 아니 생각해보니깐 1화보고 재...","[잘보고 갑니다, 어머니 아버지가 게임 소설을 봐서 상태창 개념을 음 그러냐 하고 ...","[잘보고 갑니다, 적을 때만 해도 숫자 개념도 없었고, 흑역사 같은 설정에 ㅋㅋㅋㅋ...","[잘보고 갑니다, +원형탈모, ㅋㅋㅋㅋㅋㅋㅋㅋㄱ, 와 살아남기위해 완결까지 달리셨네...","[잘보고 갑니다, 저 상황에 무슨 통행증이..., 어둠 빛 마법사\n...루미노스에..."


In [ ]:
best_data=pd.concat([Data, best_test], 1)

<ipython-input-75-e3eb2146ef37>:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  best_data=pd.concat([Data, best_test], 1)


In [ ]:
best_data.to_excel('bestnovel_data.xlsx', index=False)